In [2]:
import numpy as np
import imagej
import boto3
import tempfile
import os
import scyjava as sj
import time
import io
import pandas as pd
import re
import imageio

def generate_well_label(row, col):
    row_letter = chr(64 + row)  # Convert row number to letter (1 -> A, 2 -> B, ..., 16 -> P)
    col_str = f"{col:02d}"  # Format column as two-digit (1 -> 01, 10 -> 10)
    return f"{row_letter}{col_str}"


def modify_imagepath(filepath):
    parts = filepath.split('/')
    if 'Images' in parts:
        images_index = parts.index('Images')
        parts[images_index] = 'ImagesStacked'
        return '/'.join(parts)
    else:
        return filepath

#s3_client= boto3.client('s3')
# initialize ImageJ
sj.config.add_option('-Xmx25g')
ij = imagej.init('sc.fiji:fiji')
#print(f"ImageJ version: {ij.getVersion()}")



def max_projection(image_group, bucket_name, s3_client):
    # Initialize ImageJ plugins
    stack = sj.jimport("ij.plugin.ImagesToStack")()
    ZProjector = sj.jimport("ij.plugin.ZProjector")()

    images = []

    # Process each image in the group
    for image_key in image_group:
        response = s3_client.get_object(Bucket=bucket_name, Key=image_key)
        image_data = response['Body'].read()

        # Save as temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".tiff") as temp_file:
            temp_file.write(image_data)
            temp_file_path = temp_file.name

        # Open image with ImageJ
        img = ij.IJ.openImage(temp_file_path)
        images.append(img)
        os.remove(temp_file_path)

    # Stack and perform max intensity Z-projection
    stack_imps = stack.run(images)
    z_project_result = ZProjector.run(stack_imps, "max")
    z_project_np = ij.py.from_java(z_project_result)

    # Write image as bytes for S3 upload
    output_stream = io.BytesIO()
    imageio.imwrite(output_stream, np.array(z_project_np), format='tiff')
    output_stream.seek(0)

    # Upload to S3
    output_key = modify_imagepath(image_group[0])
    s3_client.upload_fileobj(output_stream, bucket_name, output_key)




[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [3]:
ij.getApp().getInfo(True)

'ImageJ2 2.16.0/1.54g; Java 17.0.14 [amd64]; 189MB of 25600MB'

In [159]:
df=pd.read_csv("work/PfizerBucketImageMetadata.csv")
df['Timestamp']=pd.to_datetime(df['Timestamp'],format='ISO8601')
df=df.sort_values(by="Timestamp")
df['Image_FileName'] = df['Tiff'].str.split('/').str[-1]
df['Image_PathName'] = df['Tiff'].str.rsplit('/', n=1).str[0]
df = df[['ChannelName','ChannelID', 'Image_FileName', 'Image_PathName','FieldID','PlaneID','PlateID','Row','Col','Timestamp']]

/tmp/ipykernel_120/787937202.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("work/PfizerBucketImageMetadata.csv")
/tmp/ipykernel_120/787937202.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Timestamp']=pd.to_datetime(df['Timestamp'],format='ISO8601')


In [160]:
CP=['GCOE24010961',
 'GCOE24010962',
 'GCOE24010963',
 'GCOE24010964',
 'GCOE24010965',
 'GCOE24010966',
 'GCOE24003446',
 'GCOE24003447',
 'GCOE24003448',
 'GCOE24003449',
 'GCOE24003450',
 'GCOE24003451']

channel_mapping = {1: 'Hoechst', 2: 'Alexa488', 3: 'Cy3', 4: 'Cy5'}

In [ ]:
stack_df={}

for plate in CP:
    sub=df[df.PlateID == plate] 
    for i in range(0, len(sub), 12):
    chunk=sub.iloc[i: i +12]
    for j in range(4):
            # Iterate for 4 channels
            # we recuperate each channel in the order, they are ordered by field and the final image will have the same name as the first of the Planes
            # this is for easier data wrangling as we then just take Planes == 1 from the original table
            image_group = [
                chunk.iloc[j].Image_PathName + "/" + chunk.iloc[j].Image_FileName,
                chunk.iloc[j + 4].Image_PathName + "/" + chunk.iloc[j + 4].Image_FileName,
                chunk.iloc[j + 8].Image_PathName + "/" + chunk.iloc[j + 8].Image_FileName
            ]
                
            max_projection(image_group, "clientsdata", s3_client)
    
        
    print("Plate {plate} finished! Check Images in bucket")

    #extract and change infromation to then eas up the actual image processing
    df_stack=sub[sub.PlaneID==1]
    df_stack.loc[:, 'Image_PathName'] = df_stack['Image_PathName'].apply(lambda x: x.replace('Images', 'ImagesStacked'))
    df_stack.loc[:, 'ChannelName'] = df_stack['ChannelID'].apply(lambda x: channel_mapping.get(x, 'Unknown'))
    
    stack_df[plate]=df_stack

    

    
        

In [ ]:
CP_stacked = pd.concat(stack_df.values()).reset_index(drop=True)
CP_stacked.to_csv("work/PfizerCellPaintin.csv", index=False)

In [156]:
df_stack=sub[sub.PlaneID==1]

df_stack.loc[:, 'Modified_Image_PathName'] = df_stack['Image_PathName'].apply(lambda x: x.replace('Images', 'ImagesStacked'))
df_stack.loc[:, 'ChannelName'] = df_stack['ChannelID'].apply(lambda x: channel_mapping.get(x, 'Unknown'))

/tmp/ipykernel_120/1543954679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stack.loc[:, 'Modified_Image_PathName'] = df_stack['Image_PathName'].apply(lambda x: x.replace('Images', 'ImagesStacked'))


In [157]:
df_stack

,ChannelName,ChannelID,Image_FileName,Image_PathName,FieldID,PlaneID,PlateID,Row,Col,Timestamp,Modified_Image_PathName
1354669,Alexa488,2,95-1315-1-106-71-144-47-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,1,1,GCOE24010961,1,1,2024-11-21 13:30:42.999000100-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1354671,Cy5,4,95-1315-1-106-71-144-41-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,1,1,GCOE24010961,1,1,2024-11-21 13:30:43.030200100-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1354670,Cy3,3,95-1315-1-106-71-144-46-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,1,1,GCOE24010961,1,1,2024-11-21 13:30:43.155000100-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1354668,Hoechst,1,95-1315-1-106-71-144-44-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,1,1,GCOE24010961,1,1,2024-11-21 13:30:43.248600100-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1354681,Alexa488,2,92-1315-1-106-71-144-47-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,2,1,GCOE24010961,1,1,2024-11-21 13:30:44.605800100-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
...,...,...,...,...,...,...,...,...,...,...,...
1357236,Hoechst,1,86-1338-1-106-71-144-44-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,8,1,GCOE24010961,1,24,2024-11-21 15:08:38.224237700-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1357249,Alexa488,2,87-1338-1-106-71-144-47-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,9,1,GCOE24010961,1,24,2024-11-21 15:08:39.581437700-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1357251,Cy5,4,87-1338-1-106-71-144-41-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,9,1,GCOE24010961,1,24,2024-11-21 15:08:39.612637700-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...
1357250,Cy3,3,87-1338-1-106-71-144-46-510.tiff,Pfizer_livecellpainting_64compounds/dataset-a7...,9,1,GCOE24010961,1,24,2024-11-21 15:08:39.737437700-05:00,Pfizer_livecellpainting_64compounds/dataset-a7...


In [115]:
#s3_client= boto3.client('s3')

max_projection(image_group, "clientsdata", s3_client)

In [117]:
c_1

['Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-211-140-713-47-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-208-140-713-47-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-209-140-713-47-28.tiff']

In [122]:
c_4

['Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-211-140-713-44-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-208-140-713-44-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-209-140-713-44-28.tiff']

In [1]:
import numpy as np
import imagej
import boto3
import io
import pandas as pd
import imageio
import argparse

def modify_imagepath(filepath):
    parts = filepath.split('/')
    if 'Images' in parts:
        images_index = parts.index('Images')
        parts[images_index] = 'ImagesStacked'
        return '/'.join(parts)
    else:
        return filepath

def max_projection(image_group, bucket_name, s3_client):
    images = []

    # Load each image from S3
    for image_key in image_group:
        response = s3_client.get_object(Bucket=bucket_name, Key=image_key)
        image_data = response['Body'].read()
        image = imageio.imread(io.BytesIO(image_data))  # Load as NumPy array
        images.append(image)

    # Ensure all images have the same shape
    if not all(img.shape == images[0].shape for img in images):
        raise ValueError("All images must have the same dimensions.")

    # Compute max intensity projection
    max_projection = np.maximum.reduce(images)

    # Save as TIFF in-memory for S3 upload
    output_stream = io.BytesIO()
    imageio.imwrite(output_stream, max_projection, format='tiff')
    output_stream.seek(0)

    # Upload to S3
    output_key = modify_imagepath(image_group[0])
    s3_client.upload_fileobj(output_stream, bucket_name, output_key)

In [3]:
image_group=['Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-211-140-713-47-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-208-140-713-47-28.tiff',
 'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/Images/118-907-93-209-140-713-47-28.tiff']

In [6]:
s3_client = boto3.client('s3')
images = []
for image_key in image_group:
    response = s3_client.get_object(Bucket="clientsdata", Key=image_key)
    image_data = response['Body'].read()
    image = imageio.imread(io.BytesIO(image_data))  # Load as NumPy array
    images.append(image)

/tmp/ipykernel_136/1490661487.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(io.BytesIO(image_data))  # Load as NumPy array


In [8]:
max_projection = np.maximum.reduce(images)

In [9]:
max_projection

array([[271, 251, 230, ..., 128, 129, 129],
       [279, 260, 259, ..., 112, 119, 138],
       [257, 262, 278, ..., 129, 135, 150],
       ...,
       [497, 491, 484, ..., 133, 125, 123],
       [509, 462, 476, ..., 121, 128, 123],
       [536, 477, 500, ..., 119, 127, 128]], dtype=uint16)

In [10]:
modify_imagepath(image_group[0])

'Pfizer_livecellpainting_64compounds/dataset-d31c9fc2-b4a8-11ef-9b5e-02420a000112/ImagesStacked/118-907-93-211-140-713-47-28.tiff'

In [11]:
output_stream = io.BytesIO()
imageio.imwrite(output_stream, max_projection, format='tiff')
output_stream.seek(0)

# Upload to S3
output_key = modify_imagepath(image_group[0])
s3_client.upload_fileobj(output_stream, "clientsdata", output_key)